In [1]:
from preparing import batch2train_data, load_voc
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random

выбранных слов 35029 / 37298
Было 16200, стало 15800, 0.9753086419753086


In [2]:
data, Vocabulary = load_voc()

## Подготовка данных, для использования в модели


In [3]:
inp, lenghts, target, mask, max_target_len = batch2train_data([random.choice(data) for _ in range(2)])

In [4]:
inp.size()

torch.Size([19, 2])

In [5]:
target.size()

torch.Size([5, 2])

In [6]:
max_target_len

5

# Encoder

In [7]:
EMBEDDING_SIZE = 128

In [8]:
class Encoder(nn.Module):
    def __init__(self, embedding):
        super(Encoder, self).__init__()

        self.embedding = embedding
        self.gru = nn.GRU(EMBEDDING_SIZE, EMBEDDING_SIZE, num_layers=2,
                          dropout=0.4, bidirectional=True) # будет двухнаправленный, тоесть складывает матрицы
                                                        # с конца и с начала, так больше профита даётся

    def forward(self, input_seq, length, hidden=None):

        embed = embedding(input_seq)

        packed = pack_padded_sequence(embed, length) # это запоковаем

        output, hidden = self.gru(packed, hidden)

        output, _ = pad_packed_sequence(output) # это распаковывает

        # здесь складываются 2 прохода (с конца и с начала)
        output = output[:, :, EMBEDDING_SIZE:] + output[:, :, :EMBEDDING_SIZE]

        return output, hidden


In [9]:
embedding = nn.Embedding(Vocabulary.num_words, EMBEDDING_SIZE)
encoder = Encoder(embedding)

In [10]:
encoder(inp, lenghts)

(tensor([[[ 0.3626, -0.1495, -0.2371,  ...,  0.2184, -0.1409, -0.1260],
          [ 0.0742, -0.1199, -0.3195,  ..., -0.0956,  0.1990, -0.2521]],
 
         [[ 0.6336, -0.3067, -0.4165,  ...,  0.3262, -0.1021, -0.1057],
          [-0.1778, -0.0832, -0.1958,  ..., -0.0308, -0.0802, -0.3994]],
 
         [[ 0.5875,  0.0475, -0.5112,  ...,  0.5937,  0.1572,  0.2276],
          [-0.1604, -0.0012,  0.1265,  ..., -0.2252, -0.2972, -0.0621]],
 
         ...,
 
         [[ 0.4691,  0.0661, -0.1501,  ...,  0.4301, -0.0154,  0.1177],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[ 0.4085,  0.2760, -0.0925,  ...,  0.5364, -0.1912,  0.2373],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[ 0.0563,  0.0141, -0.1292,  ...,  0.2695, -0.2154,  0.0136],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
        grad_fn=<AddBackward0>),
 tensor([[[-0.3412, -0.0491,  0.2201,  ...,  0.1816,  0.5047,  0.2633]

похоже работает